In [ ]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import numeric_utils
from pax_utils import waveform_pax_utils
from pax_utils import waveform_utils

sys.path.append(os.path.abspath("../pax"))
from pax import core



In [ ]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    event  = None
    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

      
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue

            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
                
        ############################################################################################
        ############################################################################################
        
        left                        = event.main_s2.left
        right                       = event.main_s2.right
        jsonfilename                = os.path.dirname(zipfilename) + '/pax_info.json'
        cfg                         = event_utils.getConfig(jsonfilename)
        
        df_channels_waveforms_top   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        df_channels_waveforms_top   = waveform_utils.addEmptyChannelsToDataFrame(df_channels_waveforms_top)
        
        arr_summed_waveform_top_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_top')[left:right]
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top, event)[left:right]
        
        
        ############################################################################################
        # Check that the S2 summed waveform from the event and dataframe are equal
        ############################################################################################

        sum_evt       = np.sum(arr_summed_waveform_top_evt)
        sum_df        = np.sum(arr_summed_waveform_top_df)
        
        wf_arrs_equal = numeric_utils.compareArrays(arr_summed_waveform_top_evt, arr_summed_waveform_top_df)
        wf_sums_equal = numeric_utils.compareFloats(sum_evt, sum_df)
        
        assert(wf_arrs_equal and wf_sums_equal)
        
               
        ############################################################################################
        ############################################################################################
    
        ser = df_pkl_s2s.iloc[0]
        ser = ser[1:]
        ser = ser.as_matrix().astype(np.float32)
    
        print(ser.shape)
        
        
        ############################################################################################
        # Check that the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################
    
        arr_sum_evt = ser #.astype(np.float32)
        arr_sum_df  = df_channels_waveforms_top[:]['sum'].as_matrix().astype(np.float32)
        
        sum_arr_evt = np.sum(arr_sum_evt)
        sum_arr_df  = np.sum(arr_sum_df)
    
        len_arr_evt = arr_sum_evt.size
        len_arr_df  = arr_sum_df.size
        
        arrs_equal = numeric_utils.compareArrays(arr_sum_evt, arr_sum_df)
        
        #assert(arrs_equal)
       
        
        ############################################################################################
        ############################################################################################
        
        print()
        print("Integral of Summed Waveform Event:       " + str(sum_evt))
        print("Integral of Summed Waveform DF:          " + str(sum_df))
        print()
        print("Length of S2 Integrals Array Event:      " + str(len_arr_evt))
        print("Length of S2 Integrals Array DF:         " + str(len_arr_df))
        print()
        print("Sum of S2 Integrals over channels Event: " + str(sum_arr_evt))
        print("Sum of S2 Integrals over channels DF:    " + str(sum_arr_df))
        print()
        
        display(df_channels_waveforms_top[0:5][:])
        
        
        
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        t = type(np.zeros(0))
        t_df = type(pd.DataFrame())
        
        print(t)
        
        df_zip_merged['df_s2'] = None
        #df_zip_merged['df_s2'] = df_zip_merged['df_s2'].astype('object')
        #df_zip_merged['df_s2'] = df_zip_merged['df_s2'].astype(t)
        #df_zip_merged['df_s2'] = df_zip_merged['df_s2'].astype(t_df)
        
        #df_zip_merged['df_s2'] = pd.DataFrame()
        #df_zip_merged['df_s2'] = df_channels_waveforms_top

        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged

    

In [ ]:
import glob

####################################################################################################
####################################################################################################

#ver = 'test'
ver = datetime.datetime.now().strftime("%y%m%d%H%M")
ver = 'test'

nEventsPerFile = 2 #1000
nFilesZip      = 1 #200
nEvents        = nEventsPerFile * nFilesZip
dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
file_pkl     = 'merged/merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
lst_contents = glob.glob(dir_format)

if (nFilesZip == -1):
    
    nFilesZip    = len(lst_contents)

print()
print("PAX input files: " + str(nFilesZip))
print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged = processPklEvents(zipfilename)
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

display(df[0:1][:])
print(file_pkl)

df.to_pickle(file_pkl)

